# Problem Set # 7

### MACS 30100,  Dr. Evans

### Name : Zunda Xu ( zunda@uchicago.edu )

### Problem 1: Classifier "horse" race

As usual, we first import the packages and some functions used in the problem set.

In [21]:
# Import the package
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore") 
%matplotlib inline

Now, we import the dataset and check the basic information of the orginal dataset.

In [25]:
# Import the dataset
auto_df = pd.read_csv('Auto.csv', na_values='?')
# Check the basic information of the orginal dataset
auto_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 9 columns):
mpg             397 non-null float64
cylinders       397 non-null int64
displacement    397 non-null float64
horsepower      392 non-null float64
weight          397 non-null int64
acceleration    397 non-null float64
year            397 non-null int64
origin          397 non-null int64
name            397 non-null object
dtypes: float64(4), int64(4), object(1)
memory usage: 28.0+ KB


#### Part ( a ) : Fit a logistic regression model of mpg_high and fit the model using k-fold cross validation with k=4 folds

First of all, we clean and pre-process the dataset as required.

In [26]:
# Drop the missing value of the dataset
auto_df = auto_df.dropna()

In [27]:
# Check the head of the dataset
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [28]:
auto_df = auto_df.drop(['name'], axis = 1)
auto_df['horsepower'] = auto_df['horsepower'].convert_objects(convert_numeric = True)
auto_df['const'] = 1
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,const
0,18.0,8,307.0,130.0,3504,12.0,70,1,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,1


In [29]:
# Create the category variable mpg_high
auto_df['mpg_high'] = auto_df['mpg'].apply(lambda x: 1 if x >= auto_df['mpg'].median() else 0).astype('category')

In [30]:
X = auto_df[['const','cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
y = auto_df['mpg_high'].values

Now we fit the logistic regression model of mpg_high and fit the model using k-fold cross validation with k=4 folds

In [69]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)
General_error_0 = np.zeros(k)
General_error_1 = np.zeros(k)
General_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('When k =', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE = pd.DataFrame({'Error': (y_test != y_pred).astype(int), 
                        'y_pred': y_pred})
    error_0 = MSE[MSE['y_pred'] == 0]['Error'].mean()
    error_1 = MSE[MSE['y_pred'] == 1]['Error'].mean()
    error_all_class = MSE['Error'].mean()
    General_error_0[k_ind] = error_0
    General_error_1[k_ind] = error_1
    General_MSE[k_ind] = error_all_class
    print(classification_report(y_test, y_pred, digits=3))
    print('The error rate of mpg_high = 0 is', error_0)
    print('The error rate of mpg_high = 1 is', error_1)
    print('The MSE is', error_all_class) 
    print()
    k_ind += 1

print('The average results are as follows:')
print()
print('The average error rate of mpg_high = 0 is', General_error_0.mean())
print()
print('The average error rate of mpg_high = 1 is', General_error_1.mean())
print()
print('The average MSE for all classes is', General_MSE.mean())

When k = 0
             precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

The error rate of mpg_high = 0 is 0.057692307692307696
The error rate of mpg_high = 1 is 0.13043478260869565
The MSE is 0.09183673469387756

When k = 1
             precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

The error rate of mpg_high = 0 is 0.12244897959183673
The error rate of mpg_high = 1 is 0.08163265306122448
The MSE is 0.10204081632653061

When k = 2
             precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

The error rate of mpg_high = 0 is 0.15217

According to the results shown above, the average error rate of mpg_high = 0 is ** 0.0947 **, the average error rate of mpg_high = 1 is ** 0.1182 **, and the average MSE is ** 0.10714 **. 

#### Part ( b ) : Fit a random forest model of mpg_high 

As required, we fit a random forest model of mpg_high as follows:

In [45]:
random_forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                       oob_score=True, random_state=25)
X1 = auto_df[['cylinders', 'displacement', 'horsepower', 'weight',
              'acceleration', 'year', 'origin']].values
random_forest.fit(X1, y)
rf_pred = random_forest.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : rf_pred.T[1], 'yvals': y})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)

Now we will report the MSE of the random forest model and the MSE for each category:

In [46]:
MSE_model = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
print('The MSE of the random forest model is:',MSE_model)

The MSE of the random forest model is: 0.0714285714286


In [47]:
MSE_rf_0 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rf_0['yvals'], MSE_rf_0['ypred'])
print('The MSE of the mpg_high = 0 is:',MSE_0)

The MSE of the mpg_high = 0 is: 0.0578947368421


In [48]:
MSE_rf_1 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rf_1['yvals'], MSE_rf_1['ypred'])
print('The MSE of the mpg_high = 1 is:',MSE_1)

The MSE of the mpg_high = 1 is: 0.0841584158416


According to the report shown above, we note that the error rate for category 0 is ** 0.05789473684210526 **, the error rate for category 1 is ** 0.08415841584158416 **, and the MSE of the random forest model is ** 0.07142857142857142 **.

#### Part ( c ) : Fit a support vector machines model of mpg_high 

We fit a SVM model of mpg_high

In [61]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X1)
SVMGeneral_error_0 = np.zeros(k)
SVMGeneral_error_1 = np.zeros(k)
SVMGeneral_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X1):
    print('When k =', k_ind)
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVMGeneral_error_0[k_ind] = error_0
    SVMGeneral_error_1[k_ind] = error_1
    SVMGeneral_MSE[k_ind] = error_all_class
    print(classification_report(y_test, y_pred, digits=3))  
    print('The error rate of mpg_high = 0 is', error_0)
    print('The error rate of mpg_high = 1 is', error_1)
    print('The MSE is', error_all_class) 
    print()
    k_ind += 1

When k = 0
             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

The error rate of mpg_high = 0 is 0.0
The error rate of mpg_high = 1 is 0.552083333333
The MSE is 0.540816326531

When k = 1
             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

The error rate of mpg_high = 0 is 0.520408163265
The error rate of mpg_high = 1 is nan
The MSE is 0.520408163265

When k = 2
             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

The error rate of mpg_high = 0 is 0.53125
The error rate of mpg_high = 1 is 0.0
The MSE is 0.5

**Explanation of nan**: 

From the results shown above, we note that when k = 1, the output error rate of mpg_high = 1 is nan, which means **all predicted values are 0**.

Next we will report the average MSE of the model and the average error rate of each category.

In [66]:
SVMGeneral_error_0 = SVMGeneral_error_0[~np.isnan(SVMGeneral_error_0)]
print('The average error rate of mpg_high = 0 is', SVMGeneral_error_0.mean())
print()
SVMGeneral_error_1 = SVMGeneral_error_1[~np.isnan(SVMGeneral_error_1)]
print('The average error rate of mpg_high = 1 is', SVMGeneral_error_1.mean())
print()
print('The average MSE of the SVM model is', SVMGeneral_MSE.mean())

The average error rate of mpg_high = 0 is 0.381194110709

The average error rate of mpg_high = 1 is 0.184027777778

The average MSE of the SVM model is 0.507653061224


According to the report shown above, we note that the error rate for category 0 is ** 0.381194110709 **, the error rate for category 1 is ** 0.184027777778 **, and the MSE of the SVM model is ** 0.507653061224 **.

#### Part ( D ) :  Which of the above three models is the best predictor of mpg_high ? 

**Answer:**

We return back and take a look at the results of average model MSE, average error rate of each category again:

For the Logistic regression model:

The average error rate of mpg_high = 0 is 0.0947067070586

The average error rate of mpg_high = 1 is 0.118226649127

The average MSE of the logistic regression model is 0.107142857143

For the random forest model:

The average error rate of mpg_high = 0 is: 0.0578947368421

The average error rate of mpg_high = 1 is: 0.0841584158416

The average MSE of the random forest model is: 0.0714285714286

For the SVM model:

The average error rate of mpg_high = 0 is 0.381194110709

The average error rate of mpg_high = 1 is 0.184027777778

The average MSE of the SVM model is 0.507653061224


From the results shown above, we can clearly note that, considering whatever the average model MSE or the average error rate of each category, the Random Forest model both has the lowest value among those three models. For error rate of each category of mpg_high, the lower the error rate has, the better results the prediction has. Thus, seems the random forest model is the best predictor of the mpg_high.

While for the MSE values, we note that the MSEs in the random forest model are computed by OOB prediction while the two other models use the traditional cross-validation. If the sample size is not big enough, usually the MSE computed by OOB prediction will be smaller than the cross-validation method. Considering the sample size of our dataset is not that big, and the average MSE of logistic model is not far worse than that of the random forest model, thus comparing directly by average MSEs cannot tell us which model is better, but combine the error rate of each category and the average MSEs values, the **random forest model** seems still the best predictor of the mpg_high.